In [ ]:
import pandas as pd
import geopandas as gpd
from data.datemath import date_aggs
from viz import plot_lines

In [ ]:
routes = gpd.read_file("../../data/raw/bus_routes.geojson")
rides = pd.read_csv("../../data/interim/bus_rides.csv")

In [ ]:
panel = routes.merge(rides, on='route', how='right')

In [ ]:
plot_data = panel.groupby(['route','name','geometry'],as_index=False).agg({'rides':'sum'})
plot_data = gpd.GeoDataFrame(plot_data, crs=routes.crs)
fig = plot_lines(plot_data, color_col="rides")
fig.write_json("../../reports/replication/bus-route-rides.json")
fig.show()

In [ ]:
panel = panel.pipe(date_aggs, 'date')
plot_data = panel.groupby(['route','name','geometry','year-week'],as_index=False).agg({'rides':'sum'})
plot_data = gpd.GeoDataFrame(plot_data, crs=panel.crs)
# This json is 600MB! Maybe plotly is the wrong library for animation. Check claude.
# fig = plot_lines(plot_data, color_col="rides", animation='year-week')
# fig.write_json("../../reports/replication/bus-routes-weekly.json")
# fig.show()

In [ ]:
tracts = gpd.read_file("../../data/raw/tracts.geojson")

In [ ]:
demogs = panel.sjoin(tracts, how='left', predicate='intersects')

In [ ]:
plot_data = (demogs.groupby(['route','name','geometry','date'],as_index=False)
             .agg({'rides':'sum','TOTAL_POPULATION':'sum'})
             .groupby(['route','name','geometry'],as_index=False)
             .agg({'rides':'sum','TOTAL_POPULATION':'mean'}))


In [ ]:
plot_data = gpd.GeoDataFrame(plot_data, crs=routes.crs)
fig = plot_lines(plot_data, color_col="TOTAL_POPULATION")
fig.write_json("../../reports/replication/bus-route-pop.json")
fig.show()

In [ ]:
plot_data = plot_data.assign(rides_pc = plot_data['rides'] / plot_data['TOTAL_POPULATION'])
fig = plot_lines(plot_data, color_col="rides_pc")
fig.write_json("../../reports/replication/bus-route-rides-percapita.json")
fig.show()